In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from GMST import Hiatuses
from paths import path_results, path_samoc
from constants import spy

In [ ]:
def lin_detr_ts(da):
    fit_par = np.polyfit(da.time, da.values, 1)
    detr = da - fit_par[0]*da.time - fit_par[1]
    return detr

# Data

In [ ]:
gmst_trend_anom_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_trend_anom_ctrl.nc', decode_times=False)
gmst_trend_anom_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_trend_anom_rcp.nc' , decode_times=False)

toa_ctrl  = xr.open_dataarray(f'{path_results}/TOA/TOM_ctrl.nc')
toa_rcp   = xr.open_dataarray(f'{path_results}/TOA/TOM_rcp.nc' )
toa_detr_ctrl = lin_detr_ts(toa_ctrl)
toa_detr_rcp  = lin_detr_ts(toa_rcp )

In [ ]:
# SST indices

AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)
AMO_rcp  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_rcp.nc' , decode_times=False)

N12_ctrl = xr.open_dataarray(f'{path_results}/SST/Nino12_ctrl.nc', decode_times=False)
N34_ctrl = xr.open_dataarray(f'{path_results}/SST/Nino34_ctrl.nc', decode_times=False)
N12_rcp  = xr.open_dataarray(f'{path_results}/SST/Nino12_rcp.nc' , decode_times=False)
N34_rcp  = xr.open_dataarray(f'{path_results}/SST/Nino34_rcp.nc' , decode_times=False)

PDO_ctrl = xr.open_dataset(f'{path_results}/SST/NPac_ctrl.nc', decode_times=False)
PDO_rcp  = xr.open_dataset(f'{path_results}/SST/NPac_rcp.nc' , decode_times=False)

SOM_ctrl = xr.open_dataarray(f'{path_results}/SST/SOM_index_ctrl.nc', decode_times=False)
SOM_rcp  = xr.open_dataarray(f'{path_results}/SST/SOM_index_rcp.nc' , decode_times=False)

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_ctrl.nc'  , decode_times=False)
ctrl_A = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_ctrl.nc', decode_times=False)
ctrl_P = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_ctrl.nc' , decode_times=False)
ctrl_I = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_ctrl.nc'  , decode_times=False)
ctrl_M = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_ctrl.nc' , decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_ctrl.nc', decode_times=False)
ctrl_N = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_ctrl.nc'  , decode_times=False)

# something is still wrong in CTRL year 205
for ds in [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S, ctrl_M, ctrl_N]:
    for field in ['OHC_global', 'OHC_global_levels', 'OHC_zonal', 'OHC_zonal_levels']:
        ds[field][105] = (ds[field].sel({'time':204*365}) +
                                           ds[field].sel({'time':206*365}) )/2

rcp    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_rcp.nc'   , decode_times=False)
rcp_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_rcp.nc' , decode_times=False)
rcp_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_rcp.nc'  , decode_times=False)
rcp_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_rcp.nc'   , decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_rcp.nc'  , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_rcp.nc' , decode_times=False)
rcp_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_rcp.nc'   , decode_times=False)

In [ ]:
nT, nTOA, nOHC, nAMO, nENSO, nPDO, nSOM = 0,1,2,3,4,5,6
n = 7

f, ax = plt.subplots(n, 1, figsize=(12,15), sharex=True)
hiatuses = [item for sublist in [Hiatuses('ctrl'), Hiatuses('rcp',-1700)] for item in sublist]
for i in range(n):
    ax[i].set_position([.09, 1-(i+1)*.95/n, .89, (1/n)*.95-.01])
    ax[i].tick_params(labelsize=14)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].axvline(200, c='g', lw=.5)
    ax[i].axvline(300, c='g', lw=.5)
    for hiat in hiatuses:
        ax[i].axvspan(hiat[0],hiat[1], alpha=.2)
    
    if i in [nAMO, nPDO, nSOM, nOHC, nTOA]:
        ax[i].text(.01,0.04,'10 year running mean', transform=ax[i].transAxes, color='grey')

ax[0].text(305, 0.25, 'RCP',  fontsize=16)
ax[0].text(105, 0.25, 'CTRL', fontsize=16)
        
ax[nT].set_ylabel('trend anom. [K/dec]', fontsize=16)
ax[nT].axhline(-.07, c='k', lw=.5, ls=':')
ax[nT].axhline(-.17, c='k', lw=.5, ls=':')
ax[nT].plot(gmst_trend_anom_ctrl.time/365     , gmst_trend_anom_ctrl.trend_10*10, c='C0', ls='--')
ax[nT].plot(gmst_trend_anom_ctrl.time/365     , gmst_trend_anom_ctrl.trend_15*10, c='C0', ls='-' )
ax[nT].plot(gmst_trend_anom_rcp .time/365-1700, gmst_trend_anom_rcp .trend_10*10, c='C1', ls='--')
ax[nT].plot(gmst_trend_anom_rcp .time/365-1700, gmst_trend_anom_rcp .trend_15*10, c='C1', ls='-' )
        
ax[nAMO].set_ylabel('AMO index [K]', fontsize=16)
ax[nAMO].plot(AMO_ctrl.time/365     , AMO_ctrl.rolling(time=10, center=True).mean())
ax[nAMO].plot(AMO_rcp .time/365-1700, AMO_rcp .rolling(time=10, center=True).mean())

ax[nENSO].set_ylabel('Nino3.4 index [K]', fontsize=16)
ax[nENSO].plot(N34_ctrl.time/365     , N34_ctrl, lw=.9)
ax[nENSO].plot(N34_rcp .time/365-1700, N34_rcp , lw=.9)

ax[nPDO].set_ylabel('PDO index', fontsize=16)
ax[nPDO].plot(PDO_ctrl.time/12+100, PDO_ctrl.pcs.rolling(time=120, center=True).mean())
ax[nPDO].plot(PDO_rcp .time/12+300, PDO_rcp .pcs.rolling(time=120, center=True).mean())

ax[nSOM].set_ylabel('SOM index [K]', fontsize=16)
ax[nSOM].plot(SOM_ctrl.time/365     , SOM_ctrl.rolling(time=10, center=True).mean())
ax[nSOM].plot(SOM_rcp .time/365-1700, SOM_rcp .rolling(time=10, center=True).mean())

ax[nOHC].set_ylabel('OHC anomaly', fontsize=16)
dss = [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S, ctrl_M]
for i, ds in enumerate(dss):
        qf = np.polyfit(ds.time , ds.OHC_global, 2)
        detr = ds.OHC_global - (qf[0]*ds.time**2 + qf[1]*ds.time + qf[2])
        ax[nOHC].plot(ds.time/365,
                  (detr.rolling({'time':10}, center=True).mean())/1e21,
                  c=colors[i], label=f'{labels[i]}', lw=lws[i])
ax[nOHC].legend(ncol=6)
dss = [rcp, rcp_A, rcp_P, rcp_I, rcp_S, rcp_M]
for i, ds in enumerate(dss):
        qf = np.polyfit(ds.time , ds.OHC_global, 2)
        detr = ds.OHC_global - (qf[0]*ds.time**2 + qf[1]*ds.time + qf[2])
        ax[nOHC].plot(ds.time/365-1700,
                  (detr.rolling({'time':10}, center=True).mean())/1e21,
                  c=colors[i], label=f'{labels[i]}', lw=lws[i])
        
ax[nTOA].set_ylabel('TOA [ZJ/yr]', fontsize=16)
ax[nTOA].plot(toa_ctrl.time+100, toa_detr_ctrl.rolling(time=10, center=True).mean()*spy/1e21)
ax[nTOA].plot(toa_rcp .time+300, toa_detr_rcp .rolling(time=10, center=True).mean()*spy/1e21)

ax[-1].set_xlabel('time [years]', fontsize=16)
ax[-1].set_xticks(np.arange(100,400,20))
ax[-1].set_xlim((95,390))
f.align_ylabels()
# plt.tight_layout()
plt.savefig(f'{path_results}/REGR/trend_anom_indices')